<a href="https://colab.research.google.com/github/Ea-mjolnir/Microwave-Remote-sensing/blob/main/GEE_satelliteData_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Script for Processing sentinel-1 SAR for lodging and other indices

In [ ]:
# ✅ Install required packages
!pip install rasterio --quiet
!pip install numpy --quiet
!pip install scikit-image --quiet
!pip install scipy --quiet

# ✅ Import libraries
import rasterio
import numpy as np
from google.colab import files
import os
from skimage.registration import phase_cross_correlation
from scipy.ndimage import shift

# ✅ Upload SAR GeoTIFF files
print("📂 Upload Sentinel-1 GeoTIFFs (with VV and VH bands)")
uploaded = files.upload()

if not uploaded:
    print("❌ No files uploaded. Please upload .tif files.")
else:
    print("✅ Uploaded files:")
    for file in uploaded:
        print(f" - {file}")

# ✅ Function to calculate SAR-based indices
def calculate_sar_indices(vv, vh):
    vv = vv.astype(float)
    vh = vh.astype(float)

    with np.errstate(divide='ignore', invalid='ignore'):
        vv_vh_ratio = np.where(vh != 0, vv / vh, np.nan)
        lodging_index = vv - vh  # Lower values = more flattening (lodging)
        mean_backscatter = (vv + vh) / 2

    return {
        'VV_VH_Ratio': vv_vh_ratio,
        'Lodging_Index': lodging_index,
        'Mean_Backscatter': mean_backscatter
    }

# ✅ Process SAR GeoTIFFs
output_files = []
sar_index_names = ['VV_VH_Ratio', 'Lodging_Index', 'Mean_Backscatter']

# Reference image for alignment
ref_vv = None
ref_vh = None

for i, file_name in enumerate(uploaded.keys()):
    print(f"\n🛰️ Processing {file_name}...")

    try:
        with rasterio.open(file_name) as src:
            if src.count < 2:
                print(f"⚠️ File {file_name} does not have at least 2 bands (VV, VH). Skipping.")
                continue

            vv = src.read(1)
            vh = src.read(2)

            if i == 0:
                ref_vv = vv
                ref_vh = vh
            else:
                if vv.shape == ref_vv.shape:
                    try:
                        ref_mask = ~np.isnan(ref_vv)
                        mov_mask = ~np.isnan(vv)
                        shift_yx, _, _ = phase_cross_correlation(ref_vv, vv, reference_mask=ref_mask, moving_mask=mov_mask, upsample_factor=10)
                        vv = shift(vv, shift=shift_yx, mode='nearest')
                        vh = shift(vh, shift=shift_yx, mode='nearest')
                    except Exception as e:
                        print(f"⚠️ Alignment failed for {file_name}: {e}. Proceeding without alignment.")
                else:
                    print(f"⚠️ Skipping alignment for {file_name} due to shape mismatch.")

            sar_indices = calculate_sar_indices(vv, vh)

            meta = src.meta.copy()
            meta['count'] = 2 + len(sar_indices)

            output_file = f"SAR_Processed_{os.path.basename(file_name)}"
            with rasterio.open(output_file, 'w', **meta) as dst:
                dst.write(vv, 1)
                dst.write(vh, 2)
                for idx, name in enumerate(sar_index_names, start=1):
                    dst.write(sar_indices[name], 2 + idx)

                dst.descriptions = ('VV', 'VH', *sar_index_names)

            output_files.append(output_file)
            print(f"✅ Finished: {output_file}")

    except Exception as e:
        print(f"❌ Error processing {file_name}: {str(e)}")

# ✅ Download output
for output_file in output_files:
    print(f"⬇️ Downloading {output_file} ...")
    files.download(output_file)

print("✅ All done. Processed SAR data is ready.")


📂 Upload Sentinel-1 GeoTIFFs (with VV and VH bands)


Saving Sentinel1_Egypt_2023_Tile_8.tif to Sentinel1_Egypt_2023_Tile_8.tif
Saving Sentinel1_Egypt_2023_Tile_9.tif to Sentinel1_Egypt_2023_Tile_9.tif
✅ Uploaded files:
 - Sentinel1_Egypt_2023_Tile_8.tif
 - Sentinel1_Egypt_2023_Tile_9.tif

🛰️ Processing Sentinel1_Egypt_2023_Tile_8.tif...
✅ Finished: SAR_Processed_Sentinel1_Egypt_2023_Tile_8.tif

🛰️ Processing Sentinel1_Egypt_2023_Tile_9.tif...
⚠️ Skipping alignment for Sentinel1_Egypt_2023_Tile_9.tif due to shape mismatch.
✅ Finished: SAR_Processed_Sentinel1_Egypt_2023_Tile_9.tif
⬇️ Downloading SAR_Processed_Sentinel1_Egypt_2023_Tile_8.tif ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Downloading SAR_Processed_Sentinel1_Egypt_2023_Tile_9.tif ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All done. Processed SAR data is ready.


 *Script for processing landsat 8,9 and sentinel 2 for Vegetation Indices *

In [ ]:
# Install required packages
!pip install rasterio --quiet
!pip install numpy --quiet

# Import libraries
import rasterio
import numpy as np
from google.colab import files
import os

# Upload input GeoTIFFs
print("📂 Please upload all GeoTIFFs (e.g., Sentinel2_Egypt_2023.tif, Landsat8_Egypt_2023.tif, Landsat9_Egypt_2023.tif)")
print("✅ Select ALL files in the upload dialog to process them.")
uploaded = files.upload()

# List uploaded files for verification
print("\n📄 Uploaded files:")
for file_name in uploaded.keys():
    print(f" - {file_name}")

# Define band mappings for Sentinel-2 and Landsat
band_mappings = {
    'Sentinel2': {'Blue': 1, 'Green': 2, 'Red': 3, 'NIR': 4, 'SWIR1': 5, 'SWIR2': 6},  # B2, B3, B4, B8, B11, B12
    'Landsat': {'Blue': 1, 'Green': 2, 'Red': 3, 'NIR': 4, 'SWIR1': 5, 'SWIR2': 6}    # SR_B2, SR_B3, SR_B4, SR_B5, SR_B6, SR_B7
}

# Function to calculate vegetation indices
def calculate_indices(blue, green, red, nir, swir1, swir2):
    blue, green, red, nir, swir1, swir2 = map(lambda b: b.astype(float), [blue, green, red, nir, swir1, swir2])
    valid_mask = (nir > 0) & (red > 0) & (blue > 0) & (green > 0) & (swir1 > 0) & (swir2 > 0) & \
                 (~np.isnan(nir)) & (~np.isnan(red)) & (~np.isnan(blue)) & (~np.isnan(green)) & (~np.isnan(swir1)) & (~np.isnan(swir2))

    indices = {}

    indices['NDVI'] = np.where(valid_mask & ((nir + red) != 0), (nir - red) / (nir + red), np.nan)
    evi_denominator = nir + 6 * red - 7.5 * blue + 1
    indices['EVI'] = np.where(valid_mask & (evi_denominator != 0) & (evi_denominator > 0), 2.5 * (nir - red) / evi_denominator, np.nan)
    L = 0.5
    savi_denominator = nir + red + L
    indices['SAVI'] = np.where(valid_mask & (savi_denominator != 0), (nir - red) / savi_denominator * (1 + L), np.nan)
    indices['NDMI'] = np.where(valid_mask & ((nir + swir1) != 0), (nir - swir1) / (nir + swir1), np.nan)
    indices['GNDVI'] = np.where(valid_mask & ((nir + green) != 0), (nir - green) / (nir + green), np.nan)
    msavi_term = (2 * nir + 1) ** 2 - 8 * (nir - red)
    indices['MSAVI'] = np.where(valid_mask & (msavi_term >= 0), (2 * nir + 1 - np.sqrt(msavi_term)) / 2, np.nan)
    osavi_denominator = nir + red + 0.16
    indices['OSAVI'] = np.where(valid_mask & (osavi_denominator != 0), (nir - red) / osavi_denominator, np.nan)
    indices['NDSI'] = np.where(valid_mask & ((green + swir1) != 0), (green - swir1) / (green + swir1), np.nan)
    arvi_rb = 2 * red - blue
    arvi_denominator = nir + arvi_rb
    indices['ARVI'] = np.where(valid_mask & (arvi_denominator != 0), (nir - arvi_rb) / arvi_denominator, np.nan)
    indices['GCI'] = np.where(valid_mask & (green != 0), (nir / green) - 1, np.nan)
    indices['RVI'] = np.where(valid_mask & (red != 0), nir / red, np.nan)
    indices['DVI'] = np.where(valid_mask, nir - red, np.nan)
    indices['SR'] = np.where(valid_mask & (red != 0), nir / red, np.nan)
    tvi_base = np.where(valid_mask & ((nir + red) != 0), (nir - red) / (nir + red) + 0.5, np.nan)
    indices['TVI'] = np.where(valid_mask & (tvi_base >= 0), np.sqrt(tvi_base), np.nan)
    indices['NBR'] = np.where(valid_mask & ((nir + swir2) != 0), (nir - swir2) / (nir + swir2), np.nan)

    return indices

# Process each uploaded GeoTIFF
output_files = []
index_names = ['NDVI', 'EVI', 'SAVI', 'NDMI', 'GNDVI', 'MSAVI', 'OSAVI', 'NDSI', 'ARVI', 'GCI', 'RVI', 'DVI', 'SR', 'TVI', 'NBR']

if not uploaded:
    print("❌ No files were uploaded. Please run the cell again and upload the GeoTIFFs.")
else:
    for file_name in uploaded.keys():
        lower_name = file_name.lower()

        if any(keyword in lower_name for keyword in ['sentinel', 'sentinel2', 's2']):
            band_map = band_mappings['Sentinel2']
            sensor = 'Sentinel-2'
        elif any(keyword in lower_name for keyword in ['landsat', 'landsat8', 'landsat9']):
            band_map = band_mappings['Landsat']
            sensor = 'Landsat'
        else:
            print(f"⚠️ Warning: Could not confidently detect sensor type for '{file_name}'. Defaulting to Sentinel-2.")
            band_map = band_mappings['Sentinel2']
            sensor = 'Unknown (defaulted to Sentinel-2)'

        print(f"\n🛰️ Processing {file_name} (Detected Sensor: {sensor})...")

        try:
            with rasterio.open(file_name) as src:
                if src.count < 6:
                    print(f"⚠️ Error: {file_name} has only {src.count} bands. Expected at least 6. Skipping.")
                    continue

                # Read required bands
                blue = src.read(band_map['Blue'])
                green = src.read(band_map['Green'])
                red = src.read(band_map['Red'])
                nir = src.read(band_map['NIR'])
                swir1 = src.read(band_map['SWIR1'])
                swir2 = src.read(band_map['SWIR2'])

                meta = src.meta.copy()
                original_count = src.count
                indices = calculate_indices(blue, green, red, nir, swir1, swir2)

                meta['count'] = original_count + len(index_names)
                output_file = f"Updated_{file_name}"

                with rasterio.open(output_file, 'w', **meta) as dst:
                    for i in range(1, original_count + 1):
                        dst.write(src.read(i), i)
                    for idx, index_name in enumerate(index_names, start=1):
                        dst.write(indices[index_name], original_count + idx)

                    band_names = [src.descriptions[i] or f"Band_{i+1}" for i in range(original_count)]
                    band_names.extend(index_names)
                    dst.descriptions = tuple(band_names)

                output_files.append(output_file)
                print(f"✅ Saved updated GeoTIFF: {output_file}")

        except Exception as e:
            print(f"❌ Error processing {file_name}: {str(e)}. Skipping.")

# Download updated GeoTIFFs to PC
if output_files:
    for output_file in output_files:
        print(f"⬇️ Downloading {output_file} to your PC...")
        files.download(output_file)
else:
    print("⚠️ No GeoTIFFs were processed. Check file names and try again.")

print("✅ All processing and downloads complete. Check your PC's Downloads folder.")
